# Syft Functions

## Defining a Syft Function

## Input Policies

## Output Policies

## Testing it Locally

## Submitting it for Approval

## Checking Approval

## Executing your Function

## Downloading Results